# **Generiranje podataka upotrebom CARLA simulatora**

CARLA simulator je simulator namijenjen za istraživanje autonomne vožnje kroz obuku, izradu prototipa i validaciju modela autonomne gradske vožnje, uključujući i percepciju i kontrolu. Osim što je otvorenog koda, nudi i otvorene digitalne resurse (urbane planove, zgrade, vozila, pješaka, prometnih znakova itd.) za slobodno korištenje. 

Za izradu projekta korištena je verzija 0.9.11

Prije pokretanja koda potrebno je unutar terminala pokrenuti CARLA-u na način da se pozicioniramo unutar mape u kojoj se nalazi te ju pokrenemo sljedećom naredbom:
.\CarlaUE4.exe <br> <br>
Kada se CARLA pokrene učita se početna mapa grada, a svi dodatni aktori dodaju se zasebno. U projektu koristit će se gotove skripte:
- spawn_npc.py (za kreiranje vozila i pješaka)
- dynamic_weather.py (za promjenu vremena unutar simulatora)
<br> <br>
Obje skripte nalaze se unutar examples foldera (carla/PythonAPI/examples), te se pokreću se na način da se u terminalu pozicioniramo unutar mape examples te ih pokrenemo pomoću naredbe <python 'ime_skripte'.py>


Prvenstveno ćemo pozvati sve potrebne bibiloteke, te samu CARLA-u.

In [ ]:
import glob
import os
import sys
import math
import random
import time
import numpy as np
import cv2


try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

Sada dolazimo do koraka gdje inicijaliziramo klijenta i pozivamo mapu koja se koristi, no prvo ćemo pojasniti koje gradove i mape CARLA sadrži. <br>
CARLA se sastoji od osam gradova, a svaki od tih gradova može biti neslojevit i slojevit. U ovom kontekstu slojevi se odnose na grupirane objekte unutar karte i mogu biti:
- NONE, Buildings, Decals, Foliage, Ground, ParkedVehicles, Particles, Props, StreetLights, Walls, All

##### **Neslojevite mape**

Neslojevite mape su sljedeće:
- **Town01:** Osnovni plan grada koji se sastoji od T čvorišta.
- **Town02:** Sličan planu grada Town01, ali manji.
- **Town03:** Najsloženiji (križanja sa pet prometnih traka, kružni tok, neravnine, tunel...)
- **Town04:** Beskonačna petlja s autocestom i malim gradom.
- **Town05:** Grad kvadratne mreže s križnim čvorištima i mostom.
- **Town06:** Autoceste s mnogo izlaza i ulaza na/sa njih.
- **Town07:** Seosko okruženje s uskim cestama, štalama i gotovo nepostojanim semaforima.
- **Town10:** Gradsko okruženje sa avenijama, šetalištima i realističnijim teksurama.

##### **Slojevite mape**

Izgled slojevitih karata isti je kao i kod neslojevitih, ali podržava isključivanje (world.unload_map_layer(carla.MapLayer.Buildings)) i uključivanje (world.load_map_layer(carla.MapLayer.Buildings)) slojeva karte. <br>
Isključiti se ne mogu ceste, nogostupi, semafori i prometni znakovi. <br>
Slojevite mape identificiraju se sufiksom "_Opt", primjerice 'Town01_Opt'.

##### **Prilagođene mape**

Pored već spomenutih mapa, CARLA nudi i mogućnost izrade i uvođenja nove mape, cestovne mreže i  objekata unutar mape. 

##### **Inicijaliziranje klijenta i pozivanje mape koja se koristi**


In [ ]:
client=carla.Client('localhost', 2000)
client.set_timeout(10.0)

#slojevita mapa
#world = client.load_world('Town01')

#neslojevita mapa
world = client.load_world('Town02_Opt', carla.MapLayer.Buildings | carla.MapLayer.ParkedVehicles)
#world.unload_map_layer(carla.MapLayer.Buildings)
world.unload_map_layer(carla.MapLayer.All)

#zadana mapa - Town03
#world = client.get_world()

blueprintLibrary=world.get_blueprint_library()
spawn_points=world.get_map().get_spawn_points()

##### **Stvaranje novog vozila i postavljanje kamere za promatranje**

In [ ]:
#stvaranje novog vozila
#vehicle_bp=blueprintLibrary.find('vehicle.lincoln.mkz_2020')
vehicle_bp=blueprintLibrary.filter("model3")[0]
vehicle=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
#actor_list.append(vehicle)

#stavlja kameru za promatranje iza vozila da ga vidimo kad se stvori
spectator=world.get_spectator()
transform=carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

##### **Senzori i kamere**

U kodu koji slijedi vidimo koje senzore sadrži CARLA simulator.

In [ ]:
for bp in blueprintLibrary.filter('sensor'):
    print(bp.id);

In [ ]:
for bp in blueprintLibrary.filter('camera'):
    print(bp.id);

##### **RGB kamera**

In [ ]:
#rgb kamera
camera_bp=blueprintLibrary.find('sensor.camera.rgb')
camera_init_transform=carla.Transform(carla.Location(x=2.5, z=0.7))
camera=world.spawn_actor(camera_bp, camera_init_transform, attach_to=vehicle)

In [ ]:
#prikupljanje podataka sa rgb kamere - lokalno spremanje
camera.listen(lambda image: image.save_to_disk ('rgbCamera_novo/%d064.png'%image.frame))

In [ ]:
def camera_callback(image, data_dict):
    data_dict['image']=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [ ]:
#prikupljanje podataka
image_w =camera_bp.get_attribute("image_size_x" ).as_int()
image_h =camera_bp.get_attribute("image_size_y").as_int()
camera_data = {' image' : np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

In [ ]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera' , camera_data['image'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera' , camera_data['image']) 
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllwindows()

##### **Depth kamera**

In [ ]:
#depth kamera
camera_bp=blueprintLibrary.find('sensor.camera.depth')
camera_init_transform=carla.Transform(carla.Location(x=2.5, z=0.7))
camera=world.spawn_actor(camera_bp, camera_init_transform, attach_to=vehicle)

In [ ]:
#prikupljanje podataka sa depth kamere - lokalno spremanje
camera.listen(lambda image: image.save_to_disk ('depthCamera_novo/%d064.png'%image.frame))

In [ ]:
def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [ ]:
#prikupljanje podataka
image_w =camera_bp.get_attribute("image_size_x" ).as_int()
image_h =camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'depth_image' : np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: depth_callback(image, camera_data))

In [ ]:
cv2.namedWindow('Depth Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('Depth Camera' , camera_data['depth_image'])
cv2.waitKey(1)

while True:
    cv2.imshow('Depth Camera' , camera_data[ 'depth_image']) 
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllwindows()

##### **Semantic segmentation kamera**

In [ ]:
#semantic segmentation kamera
camera_bp=blueprintLibrary.find('sensor.camera.semantic_segmentation')
camera_init_transform=carla.Transform(carla.Location(x=2.5, z=0.7))
camera=world.spawn_actor(camera_bp, camera_init_transform, attach_to=vehicle)

In [ ]:
#prikupljanje podataka sa semantic segmentation kamere - lokalno spremanje
camera.listen(lambda image: image.save_to_disk ('semanticSegmentationCamera/%d064.png'%image.frame))

In [ ]:
def semanticSegmentation_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data) , (image.height, image.width, 4))

In [ ]:
#prikupljanje podataka
image_w =camera_bp.get_attribute("image_size_x" ).as_int()
image_h =camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'sem_image' : np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: semanticSegmentation_callback(image, camera_data))

In [ ]:
cv2.namedWindow('semantic segmentation Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('semantic segmentation Camera' , camera_data['sem_image'])
cv2.waitKey(1)

while True:
    cv2.imshow('semantic segmentation Camera' , camera_data[ 'sem_image']) 
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllwindows()

##### **Upravljanje vozilom**

In [ ]:
#autopilot
vehicle.set_autopilot(True)

##### **Prestanak rada kamere**

In [ ]:
camera.stop()

##### **Generiranje vozila unutar mape**

Vozila možemo sami dodati u mapu ukoliko iz nekog razloga ne želimo koristiti skriptu 'spawn_npc.py', primjerice kada ne želimo pješake unutar mape.

In [ ]:
for i in range(200):
    vehicle_bp = random.choice(blueprintLibrary.filter('vehicle'))
    npc=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [ ]:
for v in world.get_actors(). filter('*vehicle*'):
    v.set_autopilot(True)

##### **Cilj projekta**
Cilj projekta bio je pokazati kako se promjenom različitih parametara unutar simulatora mogu izgenerirati potpuno različiti podaci koristeći različite kamere.